In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import helium

plotdir = '../../cvxportfolio/portfolio/plots/'
datadir='../../cvxportfolio/data/'

sigmas=pd.read_csv(datadir+'sigmas.csv.gz',index_col=0,parse_dates=[0]).iloc[:,:-1]
returns=pd.read_csv(datadir+'returns.csv.gz',index_col=0,parse_dates=[0])
volumes=pd.read_csv(datadir+'volumes.csv.gz',index_col=0,parse_dates=[0]).iloc[:,:-1]

start_t="2012-01-01"
end_t="2016-12-31"

In [5]:
indices = returns.columns.intersection(sigmas.columns).intersection(volumes.columns)

In [6]:
sigmas = sigmas[indices]
returns = returns[indices]
volumes = volumes[indices]
borrow_costs = borrow_costs[indices]
dividends = dividends[indices]

In [7]:
w_b = pd.Series(index=indices, data=1)
w_b.USDOLLAR = 0.
w_b/=sum(w_b)

In [8]:
simulated_hcost = helium.HoldingCost(gamma=1.0, borrow_costs=borrow_costs, dividends=dividends)

In [9]:
simulated_tcost=helium.TransactionCost(gamma = 1., 
                                   half_spread=10E-4,
                                   nonlin_coef = 0.,  #1.,
                                   sigmas = sigmas, 
                                   nonlin_power = 1.5,
                                   volumes = volumes,
                                   asym_coef = 0.)

In [60]:
# simulated_tcost, simulated_hcost
simulator = helium.MarketSimulator(returns, costs=[simulated_hcost],
                               volumes=volumes, cash_ticker='USDOLLAR')

In [61]:
policies = [helium.PeriodicRebalance(target=w_b, period=p) for p in ['day', 'week', 'month', 'quarter', 'year']]
#policies.append(helium.Hold())

In [62]:
res = pd.DataFrame(index=['Daily'])#,'Weekly','Monthly'])#,'Quarterly','Annually','Hold'])

In [63]:
simulator.run(h_init=1E8*w_b, policy=policies[1], start_date=start_t, end_date=end_t)

INFO:root:Backtest started, from 2012-01-03 00:00:00 to 2016-12-30 00:00:00
INFO:root:Getting trades at date: 2012-01-03 00:00:00
INFO:root:Propagating portfolio at time 2012-01-03 00:00:00
INFO:root:Getting trades at date: 2012-01-04 00:00:00
INFO:root:Propagating portfolio at time 2012-01-04 00:00:00
INFO:root:Getting trades at date: 2012-01-05 00:00:00
INFO:root:Propagating portfolio at time 2012-01-05 00:00:00
INFO:root:Getting trades at date: 2012-01-06 00:00:00
INFO:root:Propagating portfolio at time 2012-01-06 00:00:00
INFO:root:Getting trades at date: 2012-01-09 00:00:00
INFO:root:Propagating portfolio at time 2012-01-09 00:00:00
INFO:root:Getting trades at date: 2012-01-10 00:00:00
INFO:root:Propagating portfolio at time 2012-01-10 00:00:00
INFO:root:Getting trades at date: 2012-01-11 00:00:00
INFO:root:Propagating portfolio at time 2012-01-11 00:00:00
INFO:root:Getting trades at date: 2012-01-12 00:00:00


[999999980000.0168]
[1013831033211.7595]
[1013308834587.4294]
[1019213610924.8107]
[1.8950453802612386e+26]
[1.901990255427115e+26]
[1.9260343339461107e+26]


INFO:root:Propagating portfolio at time 2012-01-12 00:00:00
INFO:root:Getting trades at date: 2012-01-13 00:00:00
INFO:root:Propagating portfolio at time 2012-01-13 00:00:00
INFO:root:Getting trades at date: 2012-01-17 00:00:00
INFO:root:Propagating portfolio at time 2012-01-17 00:00:00
INFO:root:Getting trades at date: 2012-01-18 00:00:00
INFO:root:Propagating portfolio at time 2012-01-18 00:00:00
INFO:root:Getting trades at date: 2012-01-19 00:00:00
INFO:root:Propagating portfolio at time 2012-01-19 00:00:00
INFO:root:Getting trades at date: 2012-01-20 00:00:00
INFO:root:Propagating portfolio at time 2012-01-20 00:00:00
INFO:root:Getting trades at date: 2012-01-23 00:00:00
INFO:root:Propagating portfolio at time 2012-01-23 00:00:00
INFO:root:Getting trades at date: 2012-01-24 00:00:00
INFO:root:Propagating portfolio at time 2012-01-24 00:00:00
INFO:root:Getting trades at date: 2012-01-25 00:00:00
INFO:root:Propagating portfolio at time 2012-01-25 00:00:00


[1.9289847522537553e+26]
[1.9379523997441822e+26]
[6.404853270683882e+54]
[6.426149482222979e+54]
[6.521698103898267e+54]
[6.568557454579675e+54]
[9.968808947529135e+111]
[9.976182698621876e+111]
[9.999438757643809e+111]


INFO:root:Getting trades at date: 2012-01-26 00:00:00
INFO:root:Propagating portfolio at time 2012-01-26 00:00:00
INFO:root:Getting trades at date: 2012-01-27 00:00:00
INFO:root:Propagating portfolio at time 2012-01-27 00:00:00
INFO:root:Getting trades at date: 2012-01-30 00:00:00
INFO:root:Propagating portfolio at time 2012-01-30 00:00:00
INFO:root:Getting trades at date: 2012-01-31 00:00:00
INFO:root:Propagating portfolio at time 2012-01-31 00:00:00
INFO:root:Getting trades at date: 2012-02-01 00:00:00
INFO:root:Propagating portfolio at time 2012-02-01 00:00:00
INFO:root:Getting trades at date: 2012-02-02 00:00:00
INFO:root:Propagating portfolio at time 2012-02-02 00:00:00
INFO:root:Getting trades at date: 2012-02-03 00:00:00
INFO:root:Propagating portfolio at time 2012-02-03 00:00:00
INFO:root:Getting trades at date: 2012-02-06 00:00:00
INFO:root:Propagating portfolio at time 2012-02-06 00:00:00


[1.0100287104035013e+112]
[1.0048290693630241e+112]
[1.0031792337126997e+226]
[9.980995406631602e+225]
[9.984120215858905e+225]
[1.0112403488272856e+226]
[1.0132550927303134e+226]
[nan]


AssertionError: 

In [36]:
res = pd.DataFrame(index=['Daily','Weekly','Monthly','Quarterly','Annually'])#,'Hold'])
for label, fund_val in [('\$100M',1E8), ('\$10B',1E10)]:
    res[label] = simulator.run_multi(1E8*w_b, start_date=start_t, end_date=end_t, policies=policies, parallel=False)

INFO:root:Backtest started, from 2012-01-03 00:00:00 to 2016-12-30 00:00:00
INFO:root:Getting trades at date: 2012-01-03 00:00:00
INFO:root:Propagating portfolio at time 2012-01-03 00:00:00
INFO:root:Getting trades at date: 2012-01-04 00:00:00
INFO:root:Propagating portfolio at time 2012-01-04 00:00:00
INFO:root:Getting trades at date: 2012-01-05 00:00:00
INFO:root:Propagating portfolio at time 2012-01-05 00:00:00
INFO:root:Getting trades at date: 2012-01-06 00:00:00
INFO:root:Propagating portfolio at time 2012-01-06 00:00:00
INFO:root:Getting trades at date: 2012-01-09 00:00:00
INFO:root:Propagating portfolio at time 2012-01-09 00:00:00
INFO:root:Getting trades at date: 2012-01-10 00:00:00
INFO:root:Propagating portfolio at time 2012-01-10 00:00:00
INFO:root:Getting trades at date: 2012-01-11 00:00:00
INFO:root:Propagating portfolio at time 2012-01-11 00:00:00
INFO:root:Getting trades at date: 2012-01-12 00:00:00
INFO:root:Propagating portfolio at time 2012-01-12 00:00:00
INFO:root:Ge

INFO:root:Propagating portfolio at time 2012-04-16 00:00:00
INFO:root:Getting trades at date: 2012-04-17 00:00:00
INFO:root:Propagating portfolio at time 2012-04-17 00:00:00
INFO:root:Getting trades at date: 2012-04-18 00:00:00
INFO:root:Propagating portfolio at time 2012-04-18 00:00:00
INFO:root:Getting trades at date: 2012-04-19 00:00:00
INFO:root:Propagating portfolio at time 2012-04-19 00:00:00
INFO:root:Getting trades at date: 2012-04-20 00:00:00
INFO:root:Propagating portfolio at time 2012-04-20 00:00:00
INFO:root:Getting trades at date: 2012-04-23 00:00:00
INFO:root:Propagating portfolio at time 2012-04-23 00:00:00
INFO:root:Getting trades at date: 2012-04-24 00:00:00
INFO:root:Propagating portfolio at time 2012-04-24 00:00:00
INFO:root:Getting trades at date: 2012-04-25 00:00:00
INFO:root:Propagating portfolio at time 2012-04-25 00:00:00
INFO:root:Getting trades at date: 2012-04-26 00:00:00
INFO:root:Propagating portfolio at time 2012-04-26 00:00:00
INFO:root:Getting trades at 

INFO:root:Getting trades at date: 2012-07-27 00:00:00
INFO:root:Propagating portfolio at time 2012-07-27 00:00:00
INFO:root:Getting trades at date: 2012-07-30 00:00:00
INFO:root:Propagating portfolio at time 2012-07-30 00:00:00
INFO:root:Getting trades at date: 2012-07-31 00:00:00
INFO:root:Propagating portfolio at time 2012-07-31 00:00:00
INFO:root:Getting trades at date: 2012-08-01 00:00:00
INFO:root:Propagating portfolio at time 2012-08-01 00:00:00
INFO:root:Getting trades at date: 2012-08-02 00:00:00
INFO:root:Propagating portfolio at time 2012-08-02 00:00:00
INFO:root:Getting trades at date: 2012-08-03 00:00:00
INFO:root:Propagating portfolio at time 2012-08-03 00:00:00
INFO:root:Getting trades at date: 2012-08-06 00:00:00
INFO:root:Propagating portfolio at time 2012-08-06 00:00:00
INFO:root:Getting trades at date: 2012-08-07 00:00:00
INFO:root:Propagating portfolio at time 2012-08-07 00:00:00
INFO:root:Getting trades at date: 2012-08-08 00:00:00
INFO:root:Propagating portfolio at

INFO:root:Getting trades at date: 2012-11-09 00:00:00
INFO:root:Propagating portfolio at time 2012-11-09 00:00:00
INFO:root:Getting trades at date: 2012-11-12 00:00:00
INFO:root:Propagating portfolio at time 2012-11-12 00:00:00
INFO:root:Getting trades at date: 2012-11-13 00:00:00
INFO:root:Propagating portfolio at time 2012-11-13 00:00:00
INFO:root:Getting trades at date: 2012-11-14 00:00:00
INFO:root:Propagating portfolio at time 2012-11-14 00:00:00
INFO:root:Getting trades at date: 2012-11-15 00:00:00
INFO:root:Propagating portfolio at time 2012-11-15 00:00:00
INFO:root:Getting trades at date: 2012-11-16 00:00:00
INFO:root:Propagating portfolio at time 2012-11-16 00:00:00
INFO:root:Getting trades at date: 2012-11-19 00:00:00
INFO:root:Propagating portfolio at time 2012-11-19 00:00:00
INFO:root:Getting trades at date: 2012-11-20 00:00:00
INFO:root:Propagating portfolio at time 2012-11-20 00:00:00
INFO:root:Getting trades at date: 2012-11-21 00:00:00
INFO:root:Propagating portfolio at

INFO:root:Getting trades at date: 2013-02-26 00:00:00
INFO:root:Propagating portfolio at time 2013-02-26 00:00:00
INFO:root:Getting trades at date: 2013-02-27 00:00:00
INFO:root:Propagating portfolio at time 2013-02-27 00:00:00
INFO:root:Getting trades at date: 2013-02-28 00:00:00
INFO:root:Propagating portfolio at time 2013-02-28 00:00:00
INFO:root:Getting trades at date: 2013-03-01 00:00:00
INFO:root:Propagating portfolio at time 2013-03-01 00:00:00
INFO:root:Getting trades at date: 2013-03-04 00:00:00
INFO:root:Propagating portfolio at time 2013-03-04 00:00:00
INFO:root:Getting trades at date: 2013-03-05 00:00:00
INFO:root:Propagating portfolio at time 2013-03-05 00:00:00
INFO:root:Getting trades at date: 2013-03-06 00:00:00
INFO:root:Propagating portfolio at time 2013-03-06 00:00:00
INFO:root:Getting trades at date: 2013-03-07 00:00:00
INFO:root:Propagating portfolio at time 2013-03-07 00:00:00
INFO:root:Getting trades at date: 2013-03-08 00:00:00
INFO:root:Propagating portfolio at

...

INFO:root:Propagating portfolio at time 2015-08-12 00:00:00
INFO:root:Getting trades at date: 2015-08-13 00:00:00
INFO:root:Propagating portfolio at time 2015-08-13 00:00:00
INFO:root:Getting trades at date: 2015-08-14 00:00:00
INFO:root:Propagating portfolio at time 2015-08-14 00:00:00
INFO:root:Getting trades at date: 2015-08-17 00:00:00
INFO:root:Propagating portfolio at time 2015-08-17 00:00:00
INFO:root:Getting trades at date: 2015-08-18 00:00:00
INFO:root:Propagating portfolio at time 2015-08-18 00:00:00
INFO:root:Getting trades at date: 2015-08-19 00:00:00
INFO:root:Propagating portfolio at time 2015-08-19 00:00:00
INFO:root:Getting trades at date: 2015-08-20 00:00:00
INFO:root:Propagating portfolio at time 2015-08-20 00:00:00
INFO:root:Getting trades at date: 2015-08-21 00:00:00
INFO:root:Propagating portfolio at time 2015-08-21 00:00:00
INFO:root:Getting trades at date: 2015-08-24 00:00:00
INFO:root:Propagating portfolio at time 2015-08-24 00:00:00
INFO:root:Getting trades at 

INFO:root:Propagating portfolio at time 2015-11-23 00:00:00
INFO:root:Getting trades at date: 2015-11-24 00:00:00
INFO:root:Propagating portfolio at time 2015-11-24 00:00:00
INFO:root:Getting trades at date: 2015-11-25 00:00:00
INFO:root:Propagating portfolio at time 2015-11-25 00:00:00
INFO:root:Getting trades at date: 2015-11-27 00:00:00
INFO:root:Propagating portfolio at time 2015-11-27 00:00:00
INFO:root:Getting trades at date: 2015-11-30 00:00:00
INFO:root:Propagating portfolio at time 2015-11-30 00:00:00
INFO:root:Getting trades at date: 2015-12-01 00:00:00
INFO:root:Propagating portfolio at time 2015-12-01 00:00:00
INFO:root:Getting trades at date: 2015-12-02 00:00:00
INFO:root:Propagating portfolio at time 2015-12-02 00:00:00
INFO:root:Getting trades at date: 2015-12-03 00:00:00
INFO:root:Propagating portfolio at time 2015-12-03 00:00:00
INFO:root:Getting trades at date: 2015-12-04 00:00:00
INFO:root:Propagating portfolio at time 2015-12-04 00:00:00
INFO:root:Getting trades at 

INFO:root:Propagating portfolio at time 2016-03-09 00:00:00
INFO:root:Getting trades at date: 2016-03-10 00:00:00
INFO:root:Propagating portfolio at time 2016-03-10 00:00:00
INFO:root:Getting trades at date: 2016-03-11 00:00:00
INFO:root:Propagating portfolio at time 2016-03-11 00:00:00
INFO:root:Getting trades at date: 2016-03-14 00:00:00
INFO:root:Propagating portfolio at time 2016-03-14 00:00:00
INFO:root:Getting trades at date: 2016-03-15 00:00:00
INFO:root:Propagating portfolio at time 2016-03-15 00:00:00
INFO:root:Getting trades at date: 2016-03-16 00:00:00
INFO:root:Propagating portfolio at time 2016-03-16 00:00:00
INFO:root:Getting trades at date: 2016-03-17 00:00:00
INFO:root:Propagating portfolio at time 2016-03-17 00:00:00
INFO:root:Getting trades at date: 2016-03-18 00:00:00
INFO:root:Propagating portfolio at time 2016-03-18 00:00:00
INFO:root:Getting trades at date: 2016-03-21 00:00:00
INFO:root:Propagating portfolio at time 2016-03-21 00:00:00
INFO:root:Getting trades at 

INFO:root:Propagating portfolio at time 2016-06-21 00:00:00
INFO:root:Getting trades at date: 2016-06-22 00:00:00
INFO:root:Propagating portfolio at time 2016-06-22 00:00:00
INFO:root:Getting trades at date: 2016-06-23 00:00:00
INFO:root:Propagating portfolio at time 2016-06-23 00:00:00
INFO:root:Getting trades at date: 2016-06-24 00:00:00
INFO:root:Propagating portfolio at time 2016-06-24 00:00:00
INFO:root:Getting trades at date: 2016-06-27 00:00:00
INFO:root:Propagating portfolio at time 2016-06-27 00:00:00
INFO:root:Getting trades at date: 2016-06-28 00:00:00
INFO:root:Propagating portfolio at time 2016-06-28 00:00:00
INFO:root:Getting trades at date: 2016-06-29 00:00:00
INFO:root:Propagating portfolio at time 2016-06-29 00:00:00
INFO:root:Getting trades at date: 2016-06-30 00:00:00
INFO:root:Propagating portfolio at time 2016-06-30 00:00:00
INFO:root:Getting trades at date: 2016-07-01 00:00:00
INFO:root:Propagating portfolio at time 2016-07-01 00:00:00
INFO:root:Getting trades at 

INFO:root:Propagating portfolio at time 2016-10-03 00:00:00
INFO:root:Getting trades at date: 2016-10-04 00:00:00
INFO:root:Propagating portfolio at time 2016-10-04 00:00:00
INFO:root:Getting trades at date: 2016-10-05 00:00:00
INFO:root:Propagating portfolio at time 2016-10-05 00:00:00
INFO:root:Getting trades at date: 2016-10-06 00:00:00
INFO:root:Propagating portfolio at time 2016-10-06 00:00:00
INFO:root:Getting trades at date: 2016-10-07 00:00:00
INFO:root:Propagating portfolio at time 2016-10-07 00:00:00
INFO:root:Getting trades at date: 2016-10-10 00:00:00
INFO:root:Propagating portfolio at time 2016-10-10 00:00:00
INFO:root:Getting trades at date: 2016-10-11 00:00:00
INFO:root:Propagating portfolio at time 2016-10-11 00:00:00
INFO:root:Getting trades at date: 2016-10-12 00:00:00
INFO:root:Propagating portfolio at time 2016-10-12 00:00:00
INFO:root:Getting trades at date: 2016-10-13 00:00:00
INFO:root:Propagating portfolio at time 2016-10-13 00:00:00
INFO:root:Getting trades at 

In [37]:
used_returns = returns.loc[(returns.index>=start_t)&(returns.index<=end_t)]
benchmark_returns = pd.Series(index=used_returns.index, data=np.dot(used_returns.values, w_b.values))

table=pd.DataFrame()

In [41]:
table[r'Active return']=res.applymap(lambda res: 100*250*(res.returns - benchmark_returns).mean()).unstack()
table[r'Active risk']= res.applymap(lambda res: np.std(benchmark_returns - res.returns)*100*np.sqrt(250)).unstack()
#table[r'Trans. costs']=res.applymap(lambda res: (res.simulator_TcostModel.sum(1)/res.v).mean() * 100 * 250).unstack()
#table[r'Turnover']= res.applymap(lambda res: res.turnover.mean()*100.*250.).unstack()

/Users/yowtzu/Google Drive/repo/helium/helium/result.py:150: RuntimeWarning: invalid value encountered in true_divide
  return pd.Series(data=val.values[1:] / val.values[:-1] - 1,


In [42]:
table_print=pd.DataFrame(table,copy=True)
table_print.ix[:,:]=table_print.ix[:,:].applymap(lambda x: r'%.2f%%'%x )

/Users/yowtzu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [43]:
print(table_print.to_latex(float_format='%.2f', escape=False).replace('%',r'\%'))

\begin{tabular}{llll}
\toprule
       &       &                                      Active return &                                        Active risk \\
\midrule
\$100M & Daily &  1102268481091884977684123396898433585932654583... &  1394271597305839850189692018417736331497253905... \\
       & Weekly &  -470446270734944885577317571709300800297599009... &  1395569168513718046585952586907803146901309400... \\
       & Monthly &  2147028713848085587482876279491012534158877056... &  1378118175924084146961321189294751130495158965... \\
       & Quarterly &  1109829959755716533918799235694548693906949252... &  1233859416185742625361465976867406631963344456... \\
       & Annually &  -639944600529392966468921347267935200019984161... &  1434960721867501229667335120906256985940284851... \\
\$10B & Daily &  1102268481091884977684123396898433585932654583... &  1394271597305839850189692018417736331497253905... \\
       & Weekly &  -470446270734944885577317571709300800297599009... &  13955691685

In [45]:
plt.figure(figsize=(8,5))
for v1 in table.index.levels[0][:]:
    x = table.loc[v1]['Trans. costs']
    y = table.loc[v1]['Active risk']
    plt.plot(np.array(x),np.array(y), 'o-', label='$%s\mathrm{%s}$'%(v1[:-1],v1[-1:]))
plt.legend(loc='upper right')
plt.xlabel('Transaction cost')
plt.ylabel('Risk')

import matplotlib.ticker as mtick
ax = plt.gca()
ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f%%'))
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f%%'))
#plt.savefig(plotdir+'basic_frontier.png')

NameError: name 'x' is not defined